# Text file to vCards

My flatmate had a very old Samsung phone. The PC Suite only let me export the contact list into PDF, which I then converted to TXT.
My goal is to parse that and get the contacts in a vCard format so that she can import them from her "new" phone (Sony Ericsson w200i).

## TXT part

In [51]:
import re

In [56]:
with open('contacts.txt', 'r') as f:
    contacts = f.read()
# first contact split
contacts = contacts.split(r'Email:')
len(contacts)

206

In [144]:
# fix contacts starting with the number of the previous contact (very specific to my txt)
for i, contact in enumerate(contacts):
    note_reg = r'Note:\t\n[+|\d]\d+\t\n'
    if re.search(note_reg, contact):
        contacts[i-1] += list(re.findall(note_reg, contact))[0]
        contact = re.sub(note_reg, '', contact)

In [104]:
def reverse_name(name):
    name = name.split(', ')
    name.reverse()
    return ' '.join(name).strip()

reverse_name('WG, Niklas')
reverse_name(', Katja')
reverse_name('Javier Carrasco')

'Javier Carrasco'

In [132]:
contacts_dict = dict()

for contact in contacts:
    name = list(re.findall('[a-zA-Z, ]+[\t|\n]+\(Storage:', contact))
    if name:
        name = re.split('[\n|\t]+', name[0])[0]
        name = reverse_name(name)

        numbers = list(re.findall('[+|\d]\d+', contact))

        if numbers:
            numbers = [number for number in numbers if len(number) > 5]
            contacts_dict[name] = numbers

In [133]:
print(len(contacts_dict))
list(contacts_dict.keys())[:3]

204


['Smartphone', 'Matthias Uralt handy', 'Niklas WG']

## vCard part

I saw that vCard files (.vcf) can have multiple contacts one after the other. I will do the same.

In [114]:
import vobject

In [142]:
with open('contacts.vcf', 'w', newline='') as vcf:
    phones = ['cell', 'home', 'work']
    for name, number in contacts_dict.items():
        card = vobject.vCard()
        o = card.add('fn')
        o.value = name

        for i, num in enumerate(iter(number)):
            o = card.add('tel')
            o.type_param = phones[i]
            o.value = num
        vcf.write(card.serialize())